Disclaimer

Unless otherwise noted, the software and content on this site is
in the public domain because it contains materials developed by
the United States Geological Survey, an agency of the United States
Department of Interior. For more information, see the official USGS
copyright policy at:

http://www.usgs.gov/visual-id/credit_usgs.html#copyright

This software and content is distributed on an "AS IS" BASIS, WITHOUT
WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. Any
dependent libraries included here are distributed under open source
(or open source-like) licenses/agreements. Appropriate license agreements
are included with each library.

In [1]:
# convert decimal lat / lon in WGS83 EPSG: 4326 to NAD83(NRS2007) UTM meters, Zone 17N EPSG:3724
# to get this to work, I had to define GDAL_DATA in my Win environment path

# csherwood@usgs.gov
# 4 November 2016
import numpy as np

# GDAL Python library
from osgeo import osr
# EXIF Reader
import exifread
# Image library
from PIL import Image

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

src_crs = 'EPSG:4326'
dst_crs = 'EPSG:3724'

# source coordinate system
srs_cs = osr.SpatialReference()
srs_cs.SetFromUserInput(src_crs)

# destination coordinate system
dst_cs = osr.SpatialReference()
dst_cs.SetFromUserInput(dst_crs)

# osr image transformation object
transform = osr.CoordinateTransformation(srs_cs, dst_cs)

# print coordinate system information
print " >> SOURCE COORDINATE REFERENCE SYSTEM: "
print srs_cs

print " >> DESTINATION COORDINATE REFERENCE SYSTEM: "
print dst_cs


 >> SOURCE COORDINATE REFERENCE SYSTEM: 
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
 >> DESTINATION COORDINATE REFERENCE SYSTEM: 
PROJCS["NAD83(NSRS2007) / UTM zone 17N",
    GEOGCS["NAD83(NSRS2007)",
        DATUM["NAD83_National_Spatial_Reference_System_2007",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6759"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4759"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["cen

In [9]:
    # I checked this transform using VDATUM for the Point 1 south below, with source:
    # Source: "WGS84(transit) - use NAD83 (see NGS's HTDP)"
    # Target: "NAVD88" using GEOID12B
    # northing and easting transforms were reproduced exactly
    # for Point 1 south, height was 1.0326 m NADV88, with vertical uncertainty of 7.6158 cm
    # deltaz = 28.7909
    # for Point 9 north, deltaz - 28.7911
    # so, using:
    deltaz = 28.7910

In [17]:
fnin = "D:\\crs\\proj\\2017_Matthew\\maitane\\2016-10-30_control_points.csv"
fnout = "D:\\crs\\proj\\2017_Matthew\\maitane\\2016-10-30_GCP_UTM.csv"

header_out = "Label, Lat (WGS84), Lon (WGS84), Alt. (m), Easting (m NAD83 UTM17N), Northing (m NAD83 UTM17N), Elev. (m NADV88)"
f=open(fnin,'r')
fo = open(fnout, 'wb')
header = f.readline()
print header
fo.write(header_out+"\n")
for line in f:
    line = line.strip()
    col = line.split(',')
    name = col[0]
    lat = float(col[1])+float(col[2])/60.+float(col[3])/3600.
    lon = -(float(col[4])+float(col[5])/60.+float(col[6])/3600.)
    ze = float(col[7])
    e,n,zt = transform.TransformPoint(lon, lat, ze)
    znavd88 = zt+deltaz
    print(name, lat, lon, ze, e, n, znavd88)
    #fo.write(name+", "+lat+", "+lon+", "+ze+", "+e+", "+n+", "+znavd88+"\n")
f.close
fo.close

Label,Lat (degrees),Lat (minutes),Lat (seconds),Lon (degrees),Lon (minutes),Lon (seconds),Z (Height above the WGS84 ellipsoide),Lat (WGS84),Lon(WGS84)

('Point 1 south', 29.683666308333333, -81.21871357222223, -27.7583, 478838.975406146, 3283753.8587192334, 1.032700000000002)
('Point2 south', 29.683818480555555, -81.21888821944445, -28.6757, 478822.10975602764, 3283770.751844614, 0.11530000000000129)
('Point3 south', 29.683665716666667, -81.21893698611112, -28.7528, 478817.35943742096, 3283753.8340433664, 0.03819999999999979)
('Point4 south', 29.683768733333334, -81.21898770833333, -28.537, 478812.47354208375, 3283765.2579078246, 0.25400000000000134)
('Point5 south', 29.683624155555556, -81.21885154166667, -28.1598, 478825.6176873772, 3283749.2132986, 0.6311999999999998)
('Point6 south', 29.683519158333333, -81.21888063333334, -27.6961, 478822.7809864215, 3283737.5845945445, 1.094899999999999)
('Point7 south', 29.683355158333335, -81.21875733333333, -27.541, 478834.6762250281, 3283719.

<function close>